In [ ]:
import os
import pandas as pd
import numpy as np
import httpx
import json
import time

In [ ]:
etherscan_api_key = 'YOUR_ETHERSCAN_API_KEY'

In [ ]:
# Get full list of qualifying addresses
sepolia_contract_deployers_all = pd.read_csv('sepolia_unique_contract_deployers_all.csv')
goerli_contract_deployers_all = pd.read_csv('goerli_unique_contract_deployers_all.csv')
mainnet_contract_deployers_all = pd.read_csv('mainnet_unique_contract_deployers_all.csv')
contract_deployers_all = sepolia_contract_deployers_all.append(goerli_contract_deployers_all).append(mainnet_contract_deployers_all)
unique_contract_deployers_all = contract_deployers_all['contract_deployers'].unique()

# Export to CSV
unique_contract_deployers_all_df = pd.DataFrame(unique_contract_deployers_all, columns=['contract_deployers'])
unique_contract_deployers_all_df.to_csv('combined_unique_contract_deployers_all.csv', index=False)

In [ ]:
addresses_query = pd.read_csv('combined_unique_contract_deployers_all.csv')
address_list = addresses_query['contract_deployers'].to_numpy()
num_addresses = len(address_list)

In [ ]:
balances_df = pd.DataFrame(columns=("account","balance"))

# Balance snapshot taken on November 15, 2022
for x in range(0, num_addresses+20, 20):
    time.sleep(0.1)
    print("Extracting Addresses: "+str(x)+" to "+str(x+19), end='\r')
    twenty_addresses = []
    for address in address_list[x:x+20]:
        try:
            twenty_addresses.append(address)
        except:
            pass
    comma_separated_addresses = ",".join(twenty_addresses)
    url = "https://api-goerli.etherscan.io/api?module=account&action=balancemulti&address="+comma_separated_addresses+"&tag=latest&apikey="+etherscan_api_key
    try:
        r = httpx.get(url)
        j = r.json()
        df = pd.DataFrame.from_dict(list(j["result"]))
        balances_df = balances_df.append(df, ignore_index=True)
    except:
        time.sleep(15)
        r = httpx.get(url)
        j = r.json()
        df = pd.DataFrame.from_dict(list(j["result"]))
        balances_df = balances_df.append(df, ignore_index=True)

display(balances_df)

# Export to CSV
balances_df.to_csv('./goerli_balances.csv', index=False)